이전에 KoNLPy를 이용하여 텍스트 데이터를 토크나이징 하는 방법 + Term frequency matrix로 만들어 저장하는 방법까지 연습하였습니다. 제가 만들고 있는 soynlp를 이용해서도 토크나이징의 전처리를 할 수 있습니다. (이 패키지를 왜 쓰고 있는지는 아래 주소로 가서 튜토리얼을 읽어보세요)

    https://github.com/lovit/soynlp

이후 데이터에 대하여 이와 같은 전처리를 하는 방법은 디테일한 설명을 하지 않고 코드를 적어둘 예정입니다. 정리하는 목적으로, WordExtractor만 이용하여 토크나이징까지 하는 과정을 기술해 봅니다. 

약 50여개의 유명한 영화들에 대하여 네이버영화에서 평점과 영화평을 모은 데이터입니다. 컬럼은

    <영화 아이디, 텍스트, 평점>
    
으로 되어 있으며 tap 구분이 되어 있습니다. 이렇게 데이터의 타입을 확인하기 위해서 

    for _ in range(3):
        print(next(f)
        
를 하시면 텍스트 데이터의 맨 위 3줄을 볼 수 있습니다. 포멧을 확인한 뒤, 데이터를 로딩합니다.

In [1]:
movie_reviews = './data/merged_movie_comments.txt'
with open(movie_reviews, encoding='utf-8') as f:
    for _ in range(3):
        print(next(f).strip())

72523	명불허전	1
72523	왠지 고사 피의중간고사보다 재미가없을듯해요 만약보게된다면실망할듯	1
72523	티아라 사랑해 ㅜ	10


load_comments() 함수의 구조는 앞서 연습한 방법과 동일합니다. 한 가지, 

    docs = [(doc[1], int(doc[2])) for doc in docs if len(doc) == 3]
    
처럼 docs = [doc for doc in docs] 를 하는 동안에 영화 아이디를 제거하고, int(doc[2])처럼 str로 되어 있는 평점을 숫자로 바꿔뒀습니다. 

In [2]:
def load_comments(fname):
    with open(fname, encoding='utf-8') as f:
        docs = [doc.strip().split('\t') for doc in f]
    docs = [(doc[1], int(doc[2])) for doc in docs if len(doc) == 3]
    texts, scores = zip(*docs)
    return texts, scores

texts, scores = load_comments(movie_reviews)

In [4]:
texts[:5], scores[:5]

(('명불허전',
  '왠지 고사 피의중간고사보다 재미가없을듯해요 만약보게된다면실망할듯',
  '티아라 사랑해 ㅜ',
  '황정음 윤시윤 지붕킥 인연 김수로 티아라지연 공부의신 인연 너무너무재미있어요',
  '기대 완전'),
 (1, 1, 10, 10, 8))

soynlp.word.WordExtractor를 이용하여 cohesion score, branching entropy와 같은 word score를 학습합니다. 

In [5]:
from soynlp.word import WordExtractor

word_extractor = WordExtractor()
word_extractor.train(texts)
word_scores = word_extractor.word_scores()

training was done. used memory 4.762 Gb
all cohesion probabilities was computed. # words = 801752
all branching entropies was computed # words = 912377
all accessor variety was computed # words = 912377


MaxScoreTokenizer에 cohesion_forward * right_branching_entropy 점수를 곱한 값을 최종 단어 점수로 여기는 tokenizer_scores dictionary를 만든 뒤, 이를 이용하여 문장을 토크나이징 합니다. 

주어진 데이터에 대해서 미리 토크나이징을 해두면 좋습니다. 큰 데이터는 토크나이징이 오래 걸리기 때문에 미리 전처리가 된 데이터를 만들어두면 이후의 작업이 효율적입니다. 우리의 연습의 경우, 아래의 raw 파일이 다음처럼 토크나이징 되어 저장되어 있습니다. 

    raw file: '.data/merged_movie_comments.txt'
    
    tokenized file: './data/merged_movie_comments_tsoynlp.txt'

In [6]:
from soynlp.tokenizer import MaxScoreTokenizer

tokenizer_scores = {word:score.cohesion_forward * score.right_branching_entropy 
                    for word, score in word_scores.items()}
tokenizer = MaxScoreTokenizer(scores = tokenizer_scores)
tokenizer.tokenize('왠지 고사 피의중간고사보다 재미가없을듯해요 만약보게된다면실망할듯')

['왠지',
 '고사',
 '피의',
 '중간',
 '고사',
 '보다',
 '재미',
 '가',
 '없을',
 '듯해요',
 '만약',
 '보게',
 '된다',
 '면',
 '실망',
 '할듯']